In [1]:
import pandas as pd
from IPython.display import display

In [2]:
# Leitura dos arquivos CSV e armazenando em DataFrames do pandas
df_perfil_eleitorado = pd.read_csv('perfil_eleitorado_2020.csv', sep=';', encoding='latin-1')
df_resultados = pd.read_csv('SP_turno_1.csv', sep=';', encoding='latin-1')

In [3]:
# Consulta 1: Encontrar o município onde o candidato Bruno Covas foi mais votado
resultado1 = df_resultados[df_resultados['NM_VOTAVEL'] == 'BRUNO COVAS'].nlargest(1, 'QT_VOTOS')
municipio_mais_votado = resultado1['NM_MUNICIPIO'].iloc[0]
print("Município onde o candidato Bruno Covas foi mais votado:", municipio_mais_votado)

Município onde o candidato Bruno Covas foi mais votado: SÃO PAULO


In [12]:
# Consulta 2: Candidato mais votado em cada município
df_resultado2 = df_resultados.groupby('NM_MUNICIPIO').apply(lambda group: group.loc[group['QT_VOTOS'].idxmax()])
df_resultado2 = df_resultado2[['NM_MUNICIPIO', 'NM_VOTAVEL', 'QT_VOTOS']]
df_resultado2.reset_index(drop=True, inplace=True)

#resultado salvo em planilha
df_resultado2.to_csv('resultado_consulta2.csv', sep=';', encoding='latin-1', index=False)


In [16]:
# Consulta 3: Perfil do eleitorado que mais votou em cada candidato
df_resultado3 = df_resultados.merge(df_perfil_eleitorado, on='NM_MUNICIPIO')
df_resultado3 = df_resultado3.groupby(['NM_VOTAVEL', 'DS_FAIXA_ETARIA', 'DS_GENERO', 'DS_GRAU_ESCOLARIDADE'])['QT_VOTOS'].max().reset_index()

#resultado salvo em planilha
df_resultado3.to_csv('resultado_consulta3.csv', sep=';', encoding='latin-1', index=False)


MemoryError: Unable to allocate 168. GiB for an array with shape (22509460772,) and data type int64

In [10]:
# Consulta 4: Total de abstenções por zona eleitoral e candidato
df_resultado4 = df_resultados.merge(df_perfil_eleitorado, on='NM_MUNICIPIO', how='left')
df_resultado4 = df_resultado4.groupby(['NR_ZONA', 'NM_VOTAVEL'])['QT_ABSTENCOES'].sum().reset_index()
df_resultado4.sort_values(['NR_ZONA', 'QT_ABSTENCOES'], ascending=[True, False], inplace=True)
# resultado salvo em planilha
df_resultado4.to_csv('Abstencoes_por_zona.csv', sep=';', encoding='latin-1', index=False)


MemoryError: Unable to allocate 1.47 TiB for an array with shape (201509969568,) and data type int64

In [5]:
# Consulta 5: Candidato mais votado em toda a eleição
candidato_mais_votado = df_resultados[df_resultados['QT_VOTOS'] == df_resultados['QT_VOTOS'].max()]
print("Candidato mais votado:", candidato_mais_votado['NM_VOTAVEL'].iloc[0])
print("Número de votos:", candidato_mais_votado['QT_VOTOS'].iloc[0])

Candidato mais votado: IGOR SOARES
Número de votos: 329


In [14]:
# Consulta 6: Total de votos por partido
total_votos_por_partido = df_resultados.groupby('NM_PARTIDO')['QT_VOTOS'].sum().reset_index()
total_votos_por_partido.sort_values('QT_VOTOS', ascending=False, inplace=True)
total_votos_por_partido.to_csv('total_votos_por_partido.csv', sep=';', encoding='latin-1', index=False)

In [20]:
#Consulta 7: filtrar total de votos por gênero para candidato a prefeito eduardo suplicy
# Filtrar resultados para candidato a prefeito Eduardo Suplicy
candidato_nome = 'EDUARDO SUPLICY'  # Certifique-se de usar o nome correto
df_candidato_prefeito = df_resultados[(df_resultados['DS_CARGO_PERGUNTA'] == 'PREFEITO') &
                                      (df_resultados['NM_VOTAVEL'].str.upper() == candidato_nome.upper())]

# Junção dos DataFrames para combinar os dados do perfil eleitoral com os resultados da votação
df_genero_votos_candidato = df_perfil_eleitorado.merge(df_candidato_prefeito, on='NM_MUNICIPIO')

# Total de votos por gênero para o candidato a prefeito Eduardo Suplicy
total_votos_por_genero_candidato = df_genero_votos_candidato.groupby('DS_GENERO')['QT_VOTOS'].sum()
total_votos_por_genero_candidato.to_csv('total_votos_por_genero_candidato.csv', sep=';', encoding='latin-1', header=True)

In [16]:
#Consulta 8: Candidatos a vereador mais votados por zona eleitoral
# Filtrar apenas os candidatos a vereador
df_vereadores = df_resultados[df_resultados['DS_CARGO_PERGUNTA'] == 'Vereador']

# Encontrar os candidatos mais votados por zona eleitoral
candidatos_mais_votados_vereadores_por_zona = df_vereadores.groupby('NR_ZONA')['NM_VOTAVEL'].apply(lambda x: x.value_counts().index[0])
print("Candidatos a vereador mais votados por zona eleitoral:\n", candidatos_mais_votados_vereadores_por_zona)
candidatos_mais_votados_vereadores_por_zona.to_csv('candidatos_mais_votados_vereadores_zona.csv', sep=';', encoding='latin-1', header=True)

Candidatos a vereador mais votados por zona eleitoral:
 NR_ZONA
1      EDUARDO SUPLICY
2                 Nulo
3               Branco
4      EDUARDO SUPLICY
5               Branco
            ...       
422               PSDB
423             Branco
424             Branco
426               Nulo
427               Nulo
Name: NM_VOTAVEL, Length: 393, dtype: object


In [7]:
#Consulta 9: Candidato a prefeito mais votado por zona eleitoral
# Filtrar apenas os candidatos a prefeito
df_prefeitos = df_resultados[df_resultados['DS_CARGO_PERGUNTA'] == 'Prefeito']

# Encontrar o candidato mais votado em cada zona eleitoral
prefeito_mais_votados_zona = df_prefeitos.groupby('NR_ZONA')['NM_VOTAVEL'].apply(lambda x: x.value_counts().index[0])
prefeito_mais_votados_zona.to_csv('prefeito_mais_votados_zona.csv', sep=';', encoding='latin-1', header=True)

In [15]:
#Consulta 10: Perfil do eleitorado dos candidatos a prefeito

# Filtrar apenas candidatos a prefeito
df_candidatos_prefeito = df_resultados[df_resultados['DS_CARGO_PERGUNTA'] == 'PREFEITO']

# Junção dos DataFrames para combinar os dados do perfil eleitoral com os resultados da votação
df_resultado10 = df_candidatos_prefeito.merge(df_perfil_eleitorado, on='NM_MUNICIPIO')

# Agrupamento por candidato e informações de perfil do eleitorado, calculando o total de votos
df_profile_votes = df_resultado10.groupby(['NM_VOTAVEL', 'DS_FAIXA_ETARIA', 'DS_GENERO', 'DS_GRAU_ESCOLARIDADE'])['QT_VOTOS'].sum().reset_index()

# Encontrar o perfil com mais votos para cada candidato
candidatos = df_profile_votes['NM_VOTAVEL'].unique()

#Lista

for candidato in candidatos:
    profile_max_votes = df_profile_votes[df_profile_votes['NM_VOTAVEL'] == candidato].nlargest(1, 'QT_VOTOS')
    
    # Salvar o resultado em uma planilha CSV
    filename = f'perfil_eleitorado_{candidato}.csv'
    profile_max_votes.to_csv(filename, sep=';', encoding='latin-1', index=False)


In [17]:
#Consulta 11: Abstenção dividida por genero 
# Merge dos DataFrames para combinar informações de perfil eleitoral com resultados
df_joined = df_resultados.merge(df_perfil_eleitorado, on='NM_MUNICIPIO', how='left')

# Calcular o total final de abstenção por gênero em toda a eleição
total_abstencao_genero = df_joined.groupby('DS_GENERO')['QT_ABSTENCOES'].sum()

# Imprimir o total final de abstenção por gênero
print("Total final de abstenção por gênero em toda a eleição:\n", total_abstencao_genero)
total_abstencao_genero.to_csv('total_abstencao_genero.csv', sep=';', encoding='latin-1', header=True)

MemoryError: Unable to allocate 1.47 TiB for an array with shape (201509969568,) and data type int64